In [1]:
# Imports
from uuid import uuid4
from os import remove
from os.path import join
from tempfile import gettempdir
import numpy as np

import pandas as pd
from aequilibrae.project.database_connection import database_connection
from aequilibrae.utils.db_utils import read_and_close

from aequilibrae.transit import Transit
from aequilibrae.paths import TrafficAssignment, TrafficClass, Graph
from aequilibrae.utils.create_example import create_example
from aequilibrae.matrix import AequilibraeMatrix
from aequilibrae.project import Project


# TODO:
# 1. Figure out the capacity vector and ordering etc.
# 2. Update reference files so tests work
# 3. Build PC, understand how venv's and wsl system works from ground up, ubuntu 22.04, 
# 4. Remove this notebook from git.

# Rewrite TODO's!

# CHANGE DOCUMENTATION - WE HAVE TO ADD THE PRELOADS BEFORE SETTING THE ALGORITHM?

In [2]:
fldr = "/data/aequilibrae/models/coquimbo_updated"
print(fldr)
project = Project.from_path(fldr)

/data/aequilibrae/models/coquimbo_updated


In [3]:
# Build temporary updated coquimbo file

# fldr = "/data/aequilibrae/models/coquimbo_updated"
# project = create_example(fldr, "coquimbo")

# remove(join(fldr, "public_transport.sqlite"))
# dest_path = join(fldr, "gtfs_coquimbo.zip")

# data = Transit(project)

# transit = data.new_gtfs_builder(agency="Lisanco", file_path=dest_path)

# transit.load_date("2016-04-13")

# # Now we execute the map matching to find the real paths.
# # Depending on the GTFS size, this process can be really time-consuming.
# transit.set_allow_map_match(True)
# transit.map_match()

# # Finally, we save our GTFS into our model.
# transit.save_to_disk()

In [4]:
# Attempt to build preload with new pce column

project.network.build_graphs()
project.activate()

g = project.network.graphs["c"]
g.set_skimming(["travel_time"])
g.set_graph("travel_time")
g.set_blocked_centroid_flows(False)
g.graph["capacity"] = 500
g.graph["travel_time"] = g.graph["distance"] / 50


inclusion_cond = "start"

/home/ubuntu/git/aequilibrae/aequilibrae/project/network/network.py:343: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = pd.read_sql(sql, conn).fillna(value=np.nan)


In [5]:
n_zones = g.centroids.shape[0]
matrix = AequilibraeMatrix()
matrix.create_empty(zones=n_zones, matrix_names=["car"])
matrix.index = g.centroids

matrix.matrices[:, :, 0] = 5  # 5 trips from each OD should cause a little bit of congestion
matrix.computational_view("car")

In [6]:
transit = Transit(project)

In [7]:
def _assignment(graph, demand: AequilibraeMatrix, preload=None, name=None) -> TrafficAssignment:
    # Create assignment and set parameters
    assignment = TrafficAssignment()
    assignment.set_classes([TrafficClass("car", graph, demand)])

    if preload is not None: # Note -> preload has to be added before we set the assignment algorithm
        assignment.add_preload(preload, name=name)

    assignment.set_vdf("BPR")
    assignment.set_vdf_parameters({"alpha": 0.15, "beta": 4.0})
    assignment.set_capacity_field("capacity")
    assignment.set_time_field("travel_time")
    assignment.max_iter = 1  # AON assignment
    assignment.set_algorithm("msa")

    return assignment

def hr_to_sec(e):
    return int(e * 60 * 60)

def calc_preload(transit: Transit, start, end):
    return transit.build_pt_preload(hr_to_sec(start), hr_to_sec(end), inclusion_cond="start")

In [8]:
def test_building_pt_preload(graph: Graph, demand: AequilibraeMatrix, transit: Transit):
    """
    Check that building pt preload works correctly for a basic example from
    the coquimbo network.
    """
    preloads = [calc_preload(transit, start, end) for start, end in [(7, 8), (6.5, 8.5), (5, 10)]]

    # Check preloads increase in size as time period increases
    assert preloads[0]['preload'].sum() == 12484
    assert preloads[1]['preload'].sum() == 21264
    assert preloads[2]['preload'].sum() == 39696

    assignment = _assignment(graph, demand, preload=preloads[0])
    assert len(assignment.preloads) == len(graph.graph)

test_building_pt_preload(g, matrix, transit)

In [ ]:
def test_run(graph: Graph, demand: AequilibraeMatrix, transit: Transit):
    """Tests a full run through of pt preloading."""

    preload = calc_preload(transit, 7, 8)

    # Run non-preloaded assignment and get results
    without_assig = _assignment(graph, demand)
    without_assig.execute()
    without_res = without_assig.results()

    # Run preloaded assignment and get results
    with_assig = _assignment(graph, demand, preload=preload)
    with_assig.execute()
    with_res = with_assig.results()

    # Check that average delay increases (ie the preload has reduced speeds)
    print(with_res["Delay_factor_AB"].mean())
    print(without_res["Delay_factor_AB"].mean())
    assert with_res["Delay_factor_AB"].mean() > without_res["Delay_factor_AB"].mean()
    
test_run(g, matrix, transit)

In [ ]:
# %%
matrix.close()
project.close()